# Metropolis Hasting MCMC

The **Metropolis-Hastings algorithm**, a Markov chain Monte Carlo (MCMC) method, is used in statistics and statistical physics to generate a series of random samples from a probability distribution for which direct sampling is difficult. These samples are used to approximate the distribution and enabling tasks such as computing integrals (e.g. expected values). This algorithm is particularly useful in high dimensions. 

In this notebook we will focus on a 2-dimensional target distributions as we want to be able to illustrate how the algorithm works.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from functools import partial
import requests

#from Target import *
#from Animation_MH import *

In [ ]:
github_url = 'https://raw.githubusercontent.com/ValentinKil/MCMC/main/Target.py'
Target = requests.get(github_url).text
exec(Target)
github_url = 'https://raw.githubusercontent.com/ValentinKil/MCMC/main/Animation_MH.py'
Animation_MH = requests.get(github_url).text
exec(Animation_MH)


## Target Distributions 

Here we will focus on two relatively simple target distributions that can be find in **Target.py**. A Gaussian mixture whose modes are not too far apart **gausmix** and a ring-shaped distribution **ring**. 

### Gaussian mixture

For the mixture of Gaussian, we chose to sum 3 Gaussian distribution of dimension 2. The probability density at a point $x$ is given by:

$$
\pi(x) = \frac{1}{3} \left[ 
\mathcal{N}(x | \mu_1, \Sigma_1) + 
\mathcal{N}(x | \mu_2, \Sigma_2) + 
\mathcal{N}(x | \mu_3, \Sigma_3) 
\right]
$$

where $\mathcal{N}(x | \mu_i, \Sigma_i)$ represents the probability density function of a Gaussian distribution with mean $\mu_i$ and covariance matrix $\Sigma_i$ for the $i$^{th} component of the mixture with : 

$$
\mu_1 = \begin{pmatrix} 0 \\ 0 \end{pmatrix};~~~~~\mu_2 = \begin{pmatrix} 1 \\ 1 \end{pmatrix};~~~~~\mu_3 = \begin{pmatrix} 3 \\ 2 \end{pmatrix}
$$

and : 

$$
\Sigma_1 = \Sigma_3 =\begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix};~~~~~\Sigma_2 = \begin{pmatrix} 2 & 0.5 \\ 0.5 & 2 \end{pmatrix}.
$$

In [ ]:
# Plot of the gaussian mixture target 
X, Y = np.mgrid[-3:6:.05, -3:6:.05]
pos = np.dstack((X, Y))
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.pcolormesh(X, Y, gausmix(pos),cmap='Blues',alpha=0.8)
plt.show()

### Ring-shaped distribution

The second target distribution we will focus on is a ring-shaped distribution. This distribution mimics, in 2 dimensions, the behaviour of a simple Gaussian in very high dimension, indeed, in high dimension, the mass of a Gaussian concentrated in a ring, called the typical set. The probability density at a point $x$ is given by:

$$
\pi(x)= \exp\left(-\frac{1}{2} \left(\sqrt{x_1^2 + x_2^2} - 3\right)^2\right),
$$

In [ ]:
# Plot of the ring target 
X, Y = np.mgrid[-6:6:.05, -6:6:.05]
pos = np.dstack((X, Y))
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.pcolormesh(X, Y, ring(pos),cmap='Blues',alpha=0.8)
plt.axis('square') 
plt.show()

## Metropolis Hasting Sampler

We now want to implement a Metropolis Hasting Sampler following the algorithm explained in the lecture note. To use the animation functions provided below, the algorithm should return an np.array **sample** of size (n_iter,3), where n_iter is the number of samples we want to get. For each $i\in[[ 1,n\_iter]]$, we should let **sample[i,0:2]** be the new state proposed by the proposal distribution, and **sample[i,2]** should be True if the new state is accepted and False otherwise. The function should be general enough to accept any distribution target and any proposal/update rule.

### Implementation of the sampler
Please write your function here 

In [ ]:
def MH_sampler(target,proposal,update_rule,init_state,n_iter):
    """Run the Metropolis-Hastings algorithm for n_iter iterations."""
    return sample
        

### Plot of the Markov Chain

We want to display the MCMC method nicely, the functions can be find in **Animation_MH.py**, they takes some time and probably deserve some improvement but gives good animations. 

## Different proposals 

As mentioned in the lecture notes, many different proposals can be used for Metropolis Hasting, here we focus on Random Walk and Langevin Algorithm.

## Random Walk Metropolis

In [ ]:
def RW_update_rule(curr_state,var):

def RW_proposal(curr_state,new_state):

### Gaussian mixture

Let us start by displaying the method for our Gaussian mixture distribution.

In [ ]:
#Choose parameters
var=2.8
initialPoint=np.array([3,2])
n_iter=10000


#Sampling 
partial_RW_update_rule= partial(RW_update_rule,var=var)

sample = MH_sampler(gausmix,RW_proposal,partial_RW_update_rule,initialPoint,n_iter)


In [ ]:
#Animation

xmin,xmax=-3,6
ymin,ymax=-3,6


fig, ax = plt.subplots()

background_plot(ax,target=gausmix,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax) # Add the background figure

particle = ax.scatter([], [], c='black',s=10, marker='o', alpha=1)  # Initial position with higher alpha
trail = ax.scatter([], [], c='black',s=10, marker='o', alpha=0.5)  # Particle trail with lower alpha
line, = ax.plot([], [], c='black', alpha=0.5)  # Line connecting current and previous positions

partial_init=partial(init,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax,particle=particle,trail=trail,ax=ax,square=False)

# Create the animation
partial_update= partial(update,sample=sample,particle=particle,line=line,trail=trail)
ani = FuncAnimation(fig, partial_update, frames=200, init_func=partial_init, blit=True)

# Display the animation in the notebook
HTML(ani.to_jshtml())


A good rule of thumb for choosing the step size **var** is to tune it to give an acceptance probability somewhere between 0.2 and 0.4. We can check this by approximating this probability using the Monte Carlo method over an excursion of the Markov chain. 

In [ ]:
acceptance_prob=np.cumsum(sample[:,2])[-1]/n_iter
print(acceptance_prob)

Of course, it's difficult to see from the animation that the MH algorithm gives a good approximation of the target distribution. To illustrate this, we can plot the running average $\bar{X}_n=\frac{1}{n}\sum_{i=1}^nX_i$, where $X_i$ are the successive values returned by the algorithm. After enough MCMC iterations, this set should converge to the expected value of the target distribution.  To improve performance, we can forget about the first steps, waiting for the chain to reach the typical set where the mass is concentrated, that's the role of **burnin**.

In [ ]:
n_iter=10000
sample1 = MH_sampler(gausmix,RW_proposal,partial_RW_update_rule,initialPoint,n_iter)
sample2 = MH_sampler(gausmix,RW_proposal,partial_RW_update_rule,initialPoint,n_iter)
sample3 = MH_sampler(gausmix,RW_proposal,partial_RW_update_rule,initialPoint,n_iter)

In [ ]:
burnin=1000

# Compute cumulative averages after burnin
Avgs = [calculate_cumulative_average(sample[burnin:]) for sample in [sample1, sample2, sample3]]

nb_gaussian = len(moy_list)
Truemean = np.cumsum(moy_list, axis=0)[-1] / nb_gaussian

fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns setup

for i in range(2):  # Assuming two parameters for simplicity
    for Avg in Avgs:
        axs[i].plot(Avg[:, i])
    axs[i].axhline(Truemean[i], color='r', label=f"true $\\mu_{i+1}$")
    axs[i].set_title(f'MCMC trace plot for $\\mu_{i+1}$')
    axs[i].set_xlabel('MCMC iterations')
    axs[i].set_ylabel(f'$\\mu_{i+1}$')
    axs[i].legend()

plt.tight_layout()
plt.show()
    

### Ring-shaped distribution 

Then with the ring distribution.

In [ ]:
#Choose parameters 
var=3.2
initialPoint=np.array([-3,0])
n_iter=10000


#Sampling
partial_RW_update_rule= partial(RW_update_rule,var=var)

sample = MH_sampler(ring,RW_proposal,partial_RW_update_rule,initialPoint,n_iter)


In [ ]:
#Animation

xmin=center[0]-2*radius
xmax=center[0]+2*radius
ymin=center[1]-2*radius
ymax=center[1]+2*radius

fig, ax = plt.subplots()

background_plot(ax,target=ring,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax) # Add the background figure

particle = ax.scatter([], [], c='black',s=10, marker='o', alpha=1)  # Initial position with higher alpha
trail = ax.scatter([], [], c='black',s=10, marker='o', alpha=0.5)  # Particle trail with lower alpha
line, = ax.plot([], [], c='black', alpha=0.5)  # Line connecting current and previous positions

partial_init=partial(init,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax,particle=particle,trail=trail,ax=ax,square=True)

# Create the animation
partial_update= partial(update,sample=sample,particle=particle,line=line,trail=trail)
ani = FuncAnimation(fig, partial_update, frames=200, init_func=partial_init, blit=True)

# Display the animation in the notebook
HTML(ani.to_jshtml())

In [ ]:
acceptance_prob=np.cumsum(sample[:,2])[-1]/n_iter
print(acceptance_prob)

In [ ]:
n_iter=20000
sample1 = MH_sampler(ring,RW_proposal,partial_RW_update_rule,initialPoint,n_iter)
sample2 = MH_sampler(ring,RW_proposal,partial_RW_update_rule,initialPoint,n_iter)
sample3 = MH_sampler(ring,RW_proposal,partial_RW_update_rule,initialPoint,n_iter)

In [ ]:
burnin=1000

# Compute cumulative averages after burnin
Avgs = [calculate_cumulative_average(sample[burnin:]) for sample in [sample1, sample2, sample3]]

fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns setup

for i in range(2):  # Assuming two parameters for simplicity
    for Avg in Avgs:
        axs[i].plot(Avg[:, i])
    axs[i].axhline(center[i], color='r', label=f"true $\\mu_{i+1}$")
    axs[i].set_title(f'MCMC trace plot for $\\mu_{i+1}$')
    axs[i].set_xlabel('MCMC iterations')
    axs[i].set_ylabel(f'$\\mu_{i+1}$')
    axs[i].legend()

plt.tight_layout()
plt.show()

## Metropolis-adjusted Langevin Algorithm

For MALA we need to compute the gradient of the log proposal. For our to target they can be found in **Target.py**.

In [ ]:
def MALA_update_rule(curr_state,delta_log,sigma):

def MALA_proposal(curr_state,new_state,delta_log,sigma):


### Gaussian mixture

Lets first try with the Gaussian mixture target

In [ ]:
#Choose paramters 
sigma=3
initialPoint=np.array([0,0])
n_iter=10000

#Sampling
partial_MALA_proposal= partial(MALA_proposal,delta_log=delta_log_gauss,sigma=sigma)
partial_MALA_update_rule= partial(MALA_update_rule,delta_log=delta_log_gauss,sigma=sigma)

sample = MH_sampler(gausmix,partial_MALA_proposal,partial_MALA_update_rule,initialPoint,n_iter)


In [ ]:
#Animation

xmin,xmax=-3,6
ymin,ymax=-3,6

fig, ax = plt.subplots()

background_plot(ax,target=gausmix,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax) # Add the background figure

particle = ax.scatter([], [], c='black',s=10, marker='o', alpha=1)  # Initial position with higher alpha
trail = ax.scatter([], [], c='black',s=10, marker='o', alpha=0.5)  # Particle trail with lower alpha
line, = ax.plot([], [], c='black', alpha=0.5)  # Line connecting current and previous positions

partial_init=partial(init,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax,particle=particle,trail=trail,ax=ax,square=False)

# Create the animation
partial_update= partial(update,sample=sample,particle=particle,line=line,trail=trail)
ani = FuncAnimation(fig, partial_update, frames=200, init_func=partial_init, blit=True)

# Display the animation in the notebook
HTML(ani.to_jshtml())

A good rule of thumb for choosing  **delta** is to tune it to give an acceptance probability somewhere between 0.4 and 0.6. We can check this by approximating this probability using the Monte Carlo method over an excursion of the Markov chain. 

In [ ]:
acceptance_prob=np.cumsum(sample[:,2])[-1]/n_iter
print(acceptance_prob)

In [ ]:
n_iter=10000
sample1 = MH_sampler(gausmix,partial_MALA_proposal,partial_MALA_update_rule,initialPoint,n_iter)
sample2 = MH_sampler(gausmix,partial_MALA_proposal,partial_MALA_update_rule,initialPoint,n_iter)
sample3 = MH_sampler(gausmix,partial_MALA_proposal,partial_MALA_update_rule,initialPoint,n_iter)

In [ ]:
burnin=2000

# Compute cumulative averages after burnin
Avgs = [calculate_cumulative_average(sample[burnin:]) for sample in [sample1, sample2, sample3]]

nb_gaussian = len(moy_list)
Truemean = np.cumsum(moy_list, axis=0)[-1] / nb_gaussian

fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns setup

for i in range(2):  # Assuming two parameters for simplicity
    for Avg in Avgs:
        axs[i].plot(Avg[:, i])
    axs[i].axhline(Truemean[i], color='r', label=f"true $\\mu_{i+1}$")
    axs[i].set_title(f'MCMC trace plot for $\\mu_{i+1}$')
    axs[i].set_xlabel('MCMC iterations')
    axs[i].set_ylabel(f'$\\mu_{i+1}$')
    axs[i].legend()

plt.tight_layout()
plt.show()
    

### Ring-shaped distribution 
And then for the ring

In [ ]:
#Choose parameters 
sigma=3.5
initialPoint=np.array([-3,0])
n_iter=10000

#Sampling

partial_MALA_proposal= partial(MALA_proposal,delta_log=delta_log_ring,sigma=sigma)
partial_MALA_update_rule= partial(MALA_update_rule,delta_log=delta_log_ring,sigma=sigma)

sample = MH_sampler(ring,partial_MALA_proposal,partial_MALA_update_rule,initialPoint,n_iter)


In [ ]:
#Animation
xmin=center[0]-2*radius
xmax=center[0]+2*radius
ymin=center[1]-2*radius
ymax=center[1]+2*radius

fig, ax = plt.subplots()

background_plot(ax,target=ring,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax) # Add the background figure

particle = ax.scatter([], [], c='black',s=10, marker='o', alpha=1)  # Initial position with higher alpha
trail = ax.scatter([], [], c='black',s=10, marker='o', alpha=0.5)  # Particle trail with lower alpha
line, = ax.plot([], [], c='black', alpha=0.5)  # Line connecting current and previous positions

partial_init=partial(init,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax,particle=particle,ax=ax,trail=trail,square=True)

# Create the animation
partial_update= partial(update,sample=sample,particle=particle,line=line,trail=trail)
ani = FuncAnimation(fig, partial_update, frames=200, init_func=partial_init, blit=True)

# Display the animation in the notebook
HTML(ani.to_jshtml())

In [ ]:
acceptance_prob=np.cumsum(sample[:,2])[-1]/n_iter
print(acceptance_prob)

In [ ]:
n_iter=20000
sample1 = MH_sampler(ring,partial_MALA_proposal,partial_MALA_update_rule,initialPoint,n_iter)
sample2 = MH_sampler(ring,partial_MALA_proposal,partial_MALA_update_rule,initialPoint,n_iter)
sample3 = MH_sampler(ring,partial_MALA_proposal,partial_MALA_update_rule,initialPoint,n_iter)

In [ ]:
burnin=1000

# Compute cumulative averages after burnin
Avgs = [calculate_cumulative_average(sample[burnin:]) for sample in [sample1, sample2, sample3]]


fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns setup

for i in range(2):  # Assuming two parameters for simplicity
    for Avg in Avgs:
        axs[i].plot(Avg[:, i])
    axs[i].axhline(center[i], color='r', label=f"true $\\mu_{i+1}$")
    axs[i].set_title(f'MCMC trace plot for $\\mu_{i+1}$')
    axs[i].set_xlabel('MCMC iterations')
    axs[i].set_ylabel(f'$\\mu_{i+1}$')
    axs[i].legend()

plt.tight_layout()
plt.show()